In [1]:
import pandas as pd
import glob
from pprint import pprint

In [2]:
files = glob.glob('../Data/Archive/GrantConnect*.xlsx')

print("Reading files...")

grants = pd.concat([pd.read_excel(f, header=2) for f in files], ignore_index=True)

print("Cleaning data...")

Reading files...
Cleaning data...


In [4]:
grants = grants[grants["Agency"] != "Australian Securities and Investments Commission"]

removal_tags = r"mission|service| care| inc|incorporated|business|technologies|national|company|school|health|pty|ltd|intl|international|college|institute|local|university|church|uca |arts |private|limited|trust|hospital|aboriginal corporat| lands* council|association|centre|federation|australia|community|cancer"

grants = grants[~grants["Recipient Name"].str.contains(removal_tags, case=False, na=True)]

councils = grants[grants["Recipient Name"].str.contains(r"council|shire|city", case=False, na=True)]

print("Finding LGAs...")

lgas = pd.read_csv("../Data/Working Data/ALGA Mail List.csv", encoding='latin1')

lgas["STATE"] = lgas["STATE"].replace({"TAS": "Tas."})
lgas.loc[lgas["COUNCIL"] == "Central Coast Council", "COUNCIL"] = lgas[lgas["COUNCIL"] == "Central Coast Council"]["COUNCIL"] + " (" + lgas[lgas['COUNCIL'] == 'Central Coast Council']['STATE'] + ")"

council_names = lgas["COUNCIL"].dropna().to_list()

Finding LGAs...


In [5]:
councils.loc[councils["Recipient Name"].str.lower().str.contains(r"central coast"), "Recipient Name"] = councils[councils["Recipient Name"].str.lower().str.contains(r"central coast")]["Recipient Name"] + " (" + councils[councils["Recipient Name"].str.lower().str.contains(r"central coast")]["Recipient State/Territory"] + ")"

In [18]:
lgas[lgas["COUNCIL"] == "Campbelltown City Council"]

,COUNCIL,POSTAL ADDRESS,CITY,STATE,P/CODE,PHONE,WEBSITE,Mayor,MAYOR_POSITION,MAYOR_FNAME,MAYOR_LNAME,EMAIL,CEO_TITLE,CEO_FNAME,CEO_LNAME,CEO_POSITION,Unnamed: 16,Unnamed: 17,Unnamed: 18
21,Campbelltown City Council,PO Box 57,CAMPBELLTOWN,NSW,2560.0,02 4645 4000,www.campbelltown.nsw.gov.au,Mayor,Mayor,George,Greiss,council@campbelltown.nsw.gov.au,Ms,Lindy,Deitz,General Manager,NaN,NaN,NaN
227,Campbelltown City Council,PO Box 1,CAMPBELLTOWN,SA,5074.0,08 8366 9222,www.campbelltown.sa.gov.au,Mayor,Mayor,Jill,Whittaker OAM,mail@campbelltown.sa.gov.au,Mr,Paul,Di Lulio,Chief Executive Officer,NaN,NaN,NaN


In [9]:
from thefuzz import process
from thefuzz import fuzz

In [10]:
def similarity(name):
    tup = process.extract(name, council_names, scorer=fuzz.partial_ratio)#, score_cutoff=100)

    if tup:
        tup = [sub for sub in tup if sub[1] == 100]

        if len(tup) == 1:
            return tup[0][0]
        else:
            return None
    else:
        return None

In [11]:
councils["Assigned LGA"] = councils["Recipient Name"].str.lower().apply(similarity)

/tmp/ipykernel_194690/3616084357.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  councils["Assigned LGA"] = councils["Recipient Name"].str.lower().apply(similarity)


In [24]:
print("Finding missing LGAs...")

import inquirer

Finding missing LGAs...


In [25]:
def request_name(name):

    names_i = sorted(council_names.copy()) 
    name = name.lower()

    t = 95

    answer = "Other"

    choices = []

    temp_name = name.replace("town", "").replace("council", "").replace("city", "").replace("shire", "").replace("municipality", "").replace("of", "").replace("corporation", "").replace("/", " ").replace("the", "").replace("valley", "").replace("mount", "").replace("regional", "").strip()

    while answer == "Other" and t > 50:

        names_i = [i for i in names_i if i not in choices]

        if t > 75:
            choices = [c for c in names_i if fuzz.partial_ratio(temp_name, c.lower()) > t]
            
            if len(choices) == 0:
                t -= 5

                print("Lowering threshold to", t)

                continue

            choices.append("Delete")

        else:
            choices = ["Delete"] + names_i

        choices.append("Other")

        answer = inquirer.prompt([inquirer.List("name", 
                                            message=f"Enter the LGA for {name}",
                                            choices=choices
                                            )])
        
        answer = answer["name"]

        t -= 10

    if answer == "Other":
        answer = inquirer.prompt([inquirer.Text("name",
                                            message=f"Enter the LGA for {name} manually"
                                            )])
        answer = answer["name"]

        while answer.lower() not in council_names:
            print("The LGA you entered is not in the list. Please enter a valid LGA.")
            answer = inquirer.prompt([inquirer.Text("name",
                                            message=f"Enter the LGA for {name} manually"
                                            )])
            answer = answer["name"]

    return answer

In [12]:
missings = councils[councils["Assigned LGA"].isnull()]

In [14]:
missings[missings["Recipient Name"].str.lower().str.contains(r"central coast")]

,Agency,Parent GA ID,GA ID,Status,Publish Date,Variation Date,Approval Date,Start Date,End Date,Value (AUD),...,Recipient Suburb,Recipient Town/City,Recipient Postcode,Recipient State/Territory,Recipient Country,Delivery State/Territory,Delivery Postcode,Delivery Country,Contact Name,Assigned LGA


In [28]:
print("Requesting missing LGAs...")

print(f"There are {missings['Recipient Name'].nunique()} missing LGAs")


Requesting missing LGAs...
There are 158 missing LGAs


In [ ]:
for name in missings["Recipient Name"].unique():
    councils.loc[councils["Recipient Name"] == name, "Assigned LGA"] = request_name(name)

#councils["Assigned LGA"] = councils.apply(lambda x: request_name(x["Recipient Name"]) if x["Assigned LGA"] is None else x["Assigned LGA"], axis=1)

print("Saving data...")

councils = councils[councils["Assigned LGA"] != "Delete"]

#councils["Assigned LGA"] = councils["Assigned LGA"].apply(lambda x: x.)

#councils.to_csv("../Data/Working Data/Grants Councils.csv", index=False)